In [ ]:
# minizinc --solver Gecode --time-limit 160000 v03.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics


#9:45



In [7]:
import pandas as pd
import seaborn as sb
import os
import numpy as np

In [8]:
variables      = ["P", "X", "distances", "scheduling", "ea", "pizzas_carried"]
var_search_ann = ["anti_first_fail", "dom_w_deg", "first_fail", "impact", "input_order", "largest", "max_regret", "most_constrained", "smallest"]
# "occurrence",

val_choice_ann = ["indomain", "indomain_max",  "indomain_min","indomain_random","indomain_reverse_split","indomain_split","indomain_split_random"]
# "indomain_median",

cols = ["var", "vsa", "vca", "initTime", "solveTime", "solutions", "variables", "propagators", "propagations", "nodes", "failures", "restarts", "peakDepth"]

model = open("v03search.mzn",'r')

base = "solve :: int_search( "

#cmd = 'minizinc searchV01.mzn -s'
#print(cmd)
#out = os.popen(cmd).read()
stringa_file = model.read().split("%*")
model.close()
#print(stringa_file[1])


In [9]:
results = pd.DataFrame(columns=cols);

In [10]:
# default

nuovo_modello = stringa_file[0] + "%*" + os.linesep + "solve minimize sum(total_travel);" + os.linesep +"%*" + stringa_file[2]
f = open("v03search.mzn", "w")
f.write(nuovo_modello)
f.close()
cmd = "minizinc --solver Gecode --time-limit 180000 v03.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics"
out = os.popen(cmd).read()
temp = [line for line in out.split("\n") if "%%%mzn-stat: " in line]
stats = {}
for k in range(3, len(cols)):
    if [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line] != []:
        stats[cols[k]] = [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line][0]
    else:
        stats[cols[k]] = np.NaN
row = {"var": "P", "vsa": "default", "vca": "default"}
row.update(stats)
results = results.append(row, ignore_index = True)
results.to_csv("resultsV03.csv")
print("Iteration DEFAULT Done.")

Iteration DEFAULT Done.


In [ ]:
# all the others

for i in variables:
    for j in var_search_ann:
        for h in val_choice_ann:
            
            print(str(variables.index(i)) +" / "+ str(len(variables)))
            print(str(var_search_ann.index(j)) +" / "+ str(len(var_search_ann)))
            print(str(val_choice_ann.index(h)) +" / "+ str(len(val_choice_ann)))
            
            nuovo_modello = stringa_file[0] + "%*" + os.linesep + base + i + ", " + j + ", " + h + ") minimize sum(total_travel);" + os.linesep +"%*" + stringa_file[2]
            
            f = open("v03search.mzn", "w")
            f.write(nuovo_modello)
            f.close()

            cmd = "minizinc --solver Gecode v03search.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics -t 180000"
            
            out = os.popen(cmd).read()
            
            temp = [line for line in out.split("\n") if "%%%mzn-stat: " in line]
            
            stats = {}
            for k in range(3, len(cols)):
                if [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line] != []:
                    stats[cols[k]] = [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line][0]
                else:
                    stats[cols[k]] = np.NaN
            
            row = {"var": i, "vsa": j, "vca": h}
            row.update(stats)
            
            results = results.append(row, ignore_index = True)

            results.to_csv("resultsV03.csv")
            
            print("Iteration Done.")

0 / 6
0 / 10
0 / 8
Iteration Done.
0 / 6
0 / 10
1 / 8
Iteration Done.
0 / 6
0 / 10
2 / 8
Iteration Done.
0 / 6
0 / 10
3 / 8
Iteration Done.
0 / 6
0 / 10
4 / 8
Iteration Done.
0 / 6
0 / 10
5 / 8
Iteration Done.
0 / 6
0 / 10
6 / 8
Iteration Done.
0 / 6
0 / 10
7 / 8


Warning, ignored search annotation: indomain_split_random


Iteration Done.
0 / 6
1 / 10
0 / 8
Iteration Done.
0 / 6
1 / 10
1 / 8
Iteration Done.
0 / 6
1 / 10
2 / 8
Iteration Done.
0 / 6
1 / 10
3 / 8
Iteration Done.
0 / 6
1 / 10
4 / 8
Iteration Done.
0 / 6
1 / 10
5 / 8
Iteration Done.
0 / 6
1 / 10
6 / 8
Iteration Done.
0 / 6
1 / 10
7 / 8


Warning, ignored search annotation: indomain_split_random


Iteration Done.
0 / 6
2 / 10
0 / 8
Iteration Done.
0 / 6
2 / 10
1 / 8
Iteration Done.
0 / 6
2 / 10
2 / 8
Iteration Done.
0 / 6
2 / 10
3 / 8
Iteration Done.
0 / 6
2 / 10
4 / 8
Iteration Done.
0 / 6
2 / 10
5 / 8
Iteration Done.
0 / 6
2 / 10
6 / 8
Iteration Done.
0 / 6
2 / 10
7 / 8


...

Iteration Done.
3 / 6
8 / 10
0 / 8
Iteration Done.
3 / 6
8 / 10
1 / 8
Iteration Done.
3 / 6
8 / 10
2 / 8
Iteration Done.
3 / 6
8 / 10
3 / 8
Iteration Done.
3 / 6
8 / 10
4 / 8
Iteration Done.
3 / 6
8 / 10
5 / 8
Iteration Done.
3 / 6
8 / 10
6 / 8
Iteration Done.
3 / 6
8 / 10
7 / 8


Warning, ignored search annotation: indomain_split_random


Iteration Done.
3 / 6
9 / 10
0 / 8
Iteration Done.
3 / 6
9 / 10
1 / 8
Iteration Done.
3 / 6
9 / 10
2 / 8
Iteration Done.
3 / 6
9 / 10
3 / 8
Iteration Done.
3 / 6
9 / 10
4 / 8
Iteration Done.
3 / 6
9 / 10
5 / 8
Iteration Done.
3 / 6
9 / 10
6 / 8
Iteration Done.
3 / 6
9 / 10
7 / 8


Warning, ignored search annotation: indomain_split_random


Iteration Done.
4 / 6
0 / 10
0 / 8
Iteration Done.
4 / 6
0 / 10
1 / 8
Iteration Done.
4 / 6
0 / 10
2 / 8
Iteration Done.
4 / 6
0 / 10
3 / 8


In [ ]:
results.head()